In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining
from ray.air.integrations.wandb import WandbLoggerCallback, setup_wandb

file_path = 'path_to_your_file/random_random_10k_games.txt'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and parse the data
total_lines = sum(1 for line in open(file_path, 'r'))
data_list = []
labels_list = []
with open(file_path, 'r') as file:
    for i, line in enumerate(file, 1):
        features, label = line.strip().split(' || ')
        features = [int(x) for x in features.split(',')]
        label = int(label)
        data_list.append(features)
        labels_list.append(label)
        
        if i % 100000 == 0:
            percentage_done = (i / total_lines) * 100
            print(f"Processed {i} lines ({percentage_done:.2f}% completed)")

print(f"Processed {total_lines} lines (100% completed)")

# Convert lists to NumPy arrays and then to PyTorch tensors
data_np = np.array(data_list, dtype=np.float32)
labels_np = np.array(labels_list, dtype=np.long)
data_tensor = torch.from_numpy(data_np)
labels_tensor = torch.from_numpy(labels_np)

# Create a TensorDataset and split it
dataset = TensorDataset(data_tensor, labels_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for the training and testing sets
batch_size = 2048
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class CustomNet(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes):
        super(CustomNet, self).__init__()
        self.layers = self._make_layers(input_size, hidden_layers, num_classes)
        self.to(device)

    def forward(self, x):
        return self.layers(x)

    def _make_layers(self, input_size, hidden_layers, num_classes):
        layers = [nn.Linear(input_size, hidden_layers[0]), nn.ReLU()]
        for i in range(len(hidden_layers) - 1):
            layers += [nn.Linear(hidden_layers[i], hidden_layers[i + 1]), nn.ReLU()]
        layers.append(nn.Linear(hidden_layers[-1], num_classes))
        return nn.Sequential(*layers)

    def reset_config(self, new_config):
        if 'hidden_layers' in new_config:
            self.update_architecture(new_config['hidden_layers'])
        return True
    
    def update_architecture(self, hidden_layers):
        """
        Adjust the architecture of the network based on the new hidden_layers configuration.
        
        Args:
        hidden_layers (list): A list containing the number of neurons in each hidden layer.
        """
        # First, extract the number of features for the input and output layers
        input_features = self.layers[0].in_features
        output_features = self.layers[-1].out_features

        # Create a new list of layers
        new_layers = [nn.Linear(input_features, hidden_layers[0]), nn.ReLU()]

        # Add the new hidden layers
        for i in range(1, len(hidden_layers)):
            new_layers.append(nn.Linear(hidden_layers[i - 1], hidden_layers[i]))
            new_layers.append(nn.ReLU())

        # Add the output layer
        new_layers.append(nn.Linear(hidden_layers[-1], output_features))

        # Update the model's layers
        self.layers = nn.Sequential(*new_layers)


def train_model(config, checkpoint_dir=None):
    wandb = setup_wandb(config, project="is-project")
    model = CustomNet(config["input_size"], config["hidden_layers"], config["num_classes"]).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    if checkpoint_dir:
        model_checkpoint = os.path.join(checkpoint_dir, "model.pth")
        if os.path.isfile(model_checkpoint):
            model.load_state_dict(torch.load(model_checkpoint))

    for epoch in range(config["num_epochs"]):
        model.train()
        running_loss = 0.0
        for batch_idx, (inputs, labels) in enumerate(train_loader, 1):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if batch_idx % 10 == 0:
                print(f'Epoch [{epoch+1}/{config["num_epochs"]}], '
                      f'Batch [{batch_idx}/{len(train_loader)}], '
                      f'Loss: {loss.item():.4f}')

        epoch_loss = running_loss / len(train_loader)
        accuracy = evaluate_model(model, test_loader)
        
        wandb.log({"epoch": epoch, "loss": epoch_loss, "accuracy": accuracy})
        tune.report(accuracy=accuracy)
        
        print(f'Epoch [{epoch+1}/{config["num_epochs"]}], '
              f'Loss: {epoch_loss:.4f}, '
              f'Accuracy: {accuracy:.4f}')

        # Save checkpoint at the end of each epoch
        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "model.pth")
            torch.save(model.state_dict(), path)

    return model

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

def mutate_layers(config):
    new_layers = config["hidden_layers"]
    if np.random.rand() < 0.5:
        # Mutation: Change number of layers
        if np.random.rand() < 0.5 and len(new_layers) > 1:
            # Remove a layer
            new_layers.pop()
        else:
            # Add a new layer with a random size
            new_layers.append(np.random.choice([32, 64, 128, 256]))
    else:
        # Mutation: Change size of an existing layer
        layer_to_change = np.random.randint(len(new_layers))
        new_layers[layer_to_change] = np.random.choice([32, 64, 128, 256])
    return {"hidden_layers": new_layers}


scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=5,
    hyperparam_mutations={
        "lr": tune.loguniform(1e-4, 1e-1),
        "hidden_layers": mutate_layers  
    }
)

analysis = tune.run(
    lambda config: train_model(config),
    name="pbt_test",
    scheduler=scheduler,
    num_samples=4,
    config={
    "lr": 0.001,  # Fixed learning rate
    "num_epochs": 10,
    "input_size": 784,
    "num_classes": 2,
    "hidden_layers": [128],  # Initial architecture
},
    callbacks=[WandbLoggerCallback(project="is-project", group="is-project", api_key="a540d30f4375fd2e181491b78a9339e8feaa53e4")]
)

top_trials = analysis.get_best_trials(metric="accuracy", mode="max", limit=5)
for idx, trial in enumerate(top_trials, start=1):
    model = CustomNet(trial.config["input_size"], trial.config["hidden_layers"], trial.config["num_classes"]).to(device)
    checkpoint = analysis.get_best_checkpoint(trial)
    model.load_state_dict(torch.load(checkpoint))
    model_save_path = f'best_model_{idx}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Model {idx} saved to {model_save_path}")